# Task 3
In this notebook you will find the sql queries necessary for the task 3.

For the completion of this task, two methods were added to the PGConnector class.

In [1]:
#External imports
import os

#Internal imports
from src.PGConnector.pgconnector import PGConnector

Establishing connection: 

(create_database not necessary if you have already created it)

In [2]:
connector = PGConnector('coin_data')
connector.create_database()
connector.connect()

INFO:src.PGConnector.pgconnector:Database coin_data already exists
INFO:src.PGConnector.pgconnector:Successfully connected to database coin_data


Execute the coin_data.sql to set up the table and insert data

In [3]:
sql_file_path = os.path.abspath(os.path.join(os.getcwd(), '..','..', 'data', 'coin_data.sql'))
connector.execute_sql_file(sql_file_path)

ERROR:src.PGConnector.pgconnector:Error executing SQL file: relation "coin_data" already exists



 ### Example queries
Query for all coin data. Data is returned as a dataframe for simplicity.

In [4]:
all_data = connector.query_coin_data("SELECT * FROM coin_data")  
print("All data:", all_data)

All data:         coin        date         price  \
0    bitcoin  2021-01-01  29022.418395   
1    bitcoin  2021-01-02  29352.126792   
2    bitcoin  2021-01-03  32163.824935   
3    bitcoin  2021-01-04  33008.226203   
4    bitcoin  2021-01-05  31515.575967   
..       ...         ...           ...   
814  cardano  2021-09-26      2.311934   
815  cardano  2021-09-27      2.212884   
816  cardano  2021-09-28      2.138879   
817  cardano  2021-09-29      2.037869   
818  cardano  2021-09-30      2.058527   

                                                  json  
0    {'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bi...  
1    {'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bi...  
2    {'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bi...  
3    {'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bi...  
4    {'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bi...  
..                                                 ...  
814  {'id': 'cardano', 'symbol': 'ada', 'name': 'Ca...  
815  {'id': 'cardano', 'symbo

C:\Users\Carla\Desktop\Carla--de-Erausquin\src\PGConnector\pgconnector.py:208: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, self.connection)


### Question 1:
 Get the average price for each coin by month


In [5]:
sql_query_1 = """
    SELECT coin, DATE_TRUNC('month', date) AS month, AVG(price) AS average_price
    FROM coin_data
    GROUP BY coin, DATE_TRUNC('month', date)
    ORDER BY coin, month;"""

In [6]:
task_1 = connector.query_coin_data(sql_query_1)  
print("Task 1 result: \n", task_1)

Task 1 result: 
         coin                     month  average_price
0    bitcoin 2021-01-01 03:00:00+00:00   34634.243110
1    bitcoin 2021-02-01 03:00:00+00:00   45897.949050
2    bitcoin 2021-03-01 03:00:00+00:00   54532.805249
3    bitcoin 2021-04-01 03:00:00+00:00   57148.237151
4    bitcoin 2021-05-01 03:00:00+00:00   47187.714790
5    bitcoin 2021-06-01 03:00:00+00:00   35973.744469
6    bitcoin 2021-07-01 03:00:00+00:00   34271.528139
7    bitcoin 2021-08-01 03:00:00+00:00   45563.673526
8    bitcoin 2021-09-01 03:00:00+00:00   46085.361452
9    cardano 2021-01-01 03:00:00+00:00       0.308016
10   cardano 2021-02-01 03:00:00+00:00       0.831580
11   cardano 2021-03-01 03:00:00+00:00       1.173164
12   cardano 2021-04-01 03:00:00+00:00       1.252436
13   cardano 2021-05-01 03:00:00+00:00       1.644268
14   cardano 2021-06-01 03:00:00+00:00       1.492185
15   cardano 2021-07-01 03:00:00+00:00       1.285418
16   cardano 2021-08-01 03:00:00+00:00       2.063807
17   cardan

C:\Users\Carla\Desktop\Carla--de-Erausquin\src\PGConnector\pgconnector.py:208: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, self.connection)


### Question 2:
 Calculate for each coin, on average, how much its price has increased after it had
dropped consecutively for more than 3 days. In the same result set include the
current market cap in USD (obtainable from the JSON-typed column). Use any time
span that you find best


In [22]:
sql_query_2 = """
WITH price_changes AS (
    SELECT coin, date, price, json->'market_data'->'market_cap'->>'usd' AS market_cap_usd, LAG(price) OVER (PARTITION BY coin ORDER BY date) AS prev_price
    FROM coin_data
),
drop_sequences AS (
    SELECT coin, date, price, market_cap_usd,
        CASE WHEN price < prev_price THEN 1 ELSE 0 END AS is_drop,
        SUM(CASE WHEN price < prev_price THEN 0 ELSE 1 END) OVER (PARTITION BY coin ORDER BY date) AS group_id
    FROM price_changes
    WHERE prev_price IS NOT NULL
),
consecutive_drops AS (
    SELECT coin, date, price, market_cap_usd, group_id, COUNT(*) OVER (PARTITION BY coin, group_id) AS drop_streak
    FROM drop_sequences
    WHERE is_drop = 1
),
next_day_prices AS (
    SELECT cd.coin, cd.date AS drop_end_date, cd.price AS drop_end_price, cd.market_cap_usd, LEAD(cd2.price) OVER (PARTITION BY cd.coin ORDER BY cd.date) AS next_day_price
    FROM consecutive_drops cd
    LEFT JOIN coin_data cd2 ON cd.coin = cd2.coin 
        AND cd2.date = cd.date + INTERVAL '1 day'
    WHERE cd.drop_streak >= 3
)
SELECT coin, AVG((next_day_price - drop_end_price) / drop_end_price * 100) AS avg_price_increase_pct, MAX(market_cap_usd) AS current_market_cap_usd
    FROM next_day_prices
    WHERE next_day_price IS NOT NULL
    GROUP BY coin
    ORDER BY avg_price_increase_pct DESC;
"""

In [23]:
task_2 = connector.query_coin_data(sql_query_2)  
print("Task 2 result: \n", task_2)

Task 2 result: 
        coin  avg_price_increase_pct current_market_cap_usd
0  ethereum                4.052652     425715131137.06134
1   cardano                4.046532       9653513519.30118
2   bitcoin               -0.847851       971174701492.849


C:\Users\Carla\Desktop\Carla--de-Erausquin\src\PGConnector\pgconnector.py:223: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, self.connection)


 Where MAX() is used because when we're grouping by coin, we want to show a representative market cap value for each cryptocurrency and I believe the most current/largest market cap is most relevant for analysis.